In [1]:
# Setting the working directory
import os
os.chdir('/Users/vishalraj/GitHub/Flight_Fare_Prediction')

In [2]:
from src.data.make_dataset import load_and_save_data_by_folder

path = 'data/raw'

# Load and save concatenated data by folder
load_and_save_data_by_folder(path)

In [3]:
import pandas as pd
df = pd.read_csv("data/interim/ATL/ATL_concatenated.csv")
print(df.head())

                              legId  searchDate  flightDate startingAirport  \
0  ee4aaff43c557e1704ebf52b8abf25ad  2022-05-13  2022-06-15             ATL   
1  284bffb121ffdafba55f69d6d89a4b7d  2022-05-13  2022-06-15             ATL   
2  8efdd07b82644146d45978f61d081f3b  2022-05-13  2022-06-15             ATL   
3  e03c8964b4feb4e206636a9c5c0015b2  2022-05-13  2022-06-15             ATL   
4  f44658d6761962cecc3eabc0a29d6ef9  2022-05-13  2022-06-15             ATL   

  destinationAirport travelDuration  isBasicEconomy  isRefundable  isNonStop  \
0                BOS        PT8H51M           False         False      False   
1                BOS       PT12H31M           False         False      False   
2                BOS        PT4H35M           False         False      False   
3                BOS        PT4H38M           False         False      False   
4                BOS        PT6H43M           False         False      False   

   totalFare  ...  segmentsArrivalTimeEpochS

In [3]:
# Extract dtypes
dtype_dict = df.dtypes.to_dict()

print(dtype_dict)

{'legId': dtype('O'), 'searchDate': dtype('O'), 'flightDate': dtype('O'), 'startingAirport': dtype('O'), 'destinationAirport': dtype('O'), 'travelDuration': dtype('O'), 'isBasicEconomy': dtype('bool'), 'isRefundable': dtype('bool'), 'isNonStop': dtype('bool'), 'totalFare': dtype('float64'), 'totalTravelDistance': dtype('float64'), 'segmentsDepartureTimeEpochSeconds': dtype('O'), 'segmentsDepartureTimeRaw': dtype('O'), 'segmentsArrivalTimeEpochSeconds': dtype('O'), 'segmentsArrivalTimeRaw': dtype('O'), 'segmentsArrivalAirportCode': dtype('O'), 'segmentsDepartureAirportCode': dtype('O'), 'segmentsAirlineName': dtype('O'), 'segmentsAirlineCode': dtype('O'), 'segmentsEquipmentDescription': dtype('O'), 'segmentsDurationInSeconds': dtype('O'), 'segmentsDistance': dtype('O'), 'segmentsCabinCode': dtype('O')}


In [2]:
from src.data.data_preprocessor import DataPreprocessor

# Create an instance of the DataPreprocessor class
preprocessor = DataPreprocessor()

# Process all folders
preprocessor.process_all_folders()

Fitting the global encoder.
Transforming the data using global encoder.
Fitting the global encoder.
Transforming the data using global encoder.
Fitting the global encoder.
Transforming the data using global encoder.
Fitting the global encoder.
Transforming the data using global encoder.


ValueError: The output of the 'cat_emb' transformer should be 2D (scipy matrix, array, or pandas DataFrame).

In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from src.models.train_model import load_and_combine_data, build_model

# Load and preprocess data
data = load_and_combine_data()

# Split the data into training, validation, and test sets
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
valid_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Build the model
model = build_model()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

# Split data into wide and deep components for training and validation
# Convert boolean columns to int
train_data['flightDate_is_weekend'] = train_data['flightDate_is_weekend'].astype(int)
valid_data['flightDate_is_weekend'] = valid_data['flightDate_is_weekend'].astype(int)
test_data['flightDate_is_weekend'] = test_data['flightDate_is_weekend'].astype(int)

# Split data into wide and deep components, and ensure they are of type float32
train_wide = train_data[['startingAirport', 'destinationAirport', 'segmentsCabinCode', 'flightDate_year', 'flightDate_month', 'flightDate_day', 'flightDate_weekday', 'flightDate_is_weekend', 'segmentsDepartureTimeRaw_hour', 'segmentsDepartureTimeRaw_minute']].values.astype('float32')
train_deep = train_data[['totalTravelDistance', 'segmentsDurationInSeconds', 'segmentsDistance']].values.astype('float32')
train_labels = train_data['totalFare'].values.astype('float32')

valid_wide = valid_data[['startingAirport', 'destinationAirport', 'segmentsCabinCode', 'flightDate_year', 'flightDate_month', 'flightDate_day', 'flightDate_weekday', 'flightDate_is_weekend', 'segmentsDepartureTimeRaw_hour', 'segmentsDepartureTimeRaw_minute']].values.astype('float32')
valid_deep = valid_data[['totalTravelDistance', 'segmentsDurationInSeconds', 'segmentsDistance']].values.astype('float32')
valid_labels = valid_data['totalFare'].values.astype('float32')

# Train the model with early stopping, reduce learning rate on plateau, and model checkpoint
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("saved_model/best_model.keras", monitor='val_loss', save_best_only=True)

history = model.fit([train_wide, train_deep], train_labels, 
                    validation_data=([valid_wide, valid_deep], valid_labels),
                    epochs=100, 
                    callbacks=[early_stop, reduce_lr, model_checkpoint])

# Save the final model to disk
model.save("models/my_final_model.keras")

# Model Evaluation
test_wide = test_data[['startingAirport', 'destinationAirport', 'segmentsCabinCode', 'flightDate_year', 'flightDate_month', 'flightDate_day', 'flightDate_weekday', 'flightDate_is_weekend', 'segmentsDepartureTimeRaw_hour', 'segmentsDepartureTimeRaw_minute']]
test_deep = test_data[['totalTravelDistance', 'segmentsDurationInSeconds', 'segmentsDistance']]
test_labels = test_data['totalFare']

# Predictions on the test set
predictions = model.predict([test_wide, test_deep])

# Calculate RMSE and MAE
rmse = tf.keras.metrics.RootMeanSquaredError()
mae = tf.keras.metrics.MeanAbsoluteError()

rmse.update_state(test_labels, predictions)
mae.update_state(test_labels, predictions)

print(f"RMSE on test set: {rmse.result().numpy()}")
print(f"MAE on test set: {mae.result().numpy()}")

Epoch 1/100
295750/295750 [==============================] - 352s 1ms/step - loss: 25743.1738 - mae: 113.1718 - mse: 25743.1738 - val_loss: 23917.2227 - val_mae: 109.3110 - val_mse: 23917.2227 - lr: 0.0010
Epoch 2/100
   146/295750 [..............................] - ETA: 5:07 - loss: 24212.4219 - mae: 110.3315 - mse: 24212.4219

/Users/vishalraj/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


295750/295750 [==============================] - 312s 1ms/step - loss: 23921.5488 - mae: 108.3392 - mse: 23921.5488 - val_loss: 23174.2969 - val_mae: 105.3933 - val_mse: 23174.2969 - lr: 0.0010
Epoch 3/100
295750/295750 [==============================] - 305s 1ms/step - loss: 23546.7129 - mae: 107.1621 - mse: 23546.7129 - val_loss: 22977.2051 - val_mae: 105.0599 - val_mse: 22977.2051 - lr: 0.0010
Epoch 4/100
295750/295750 [==============================] - 296s 999us/step - loss: 23348.7617 - mae: 106.5543 - mse: 23348.7617 - val_loss: 22696.9160 - val_mae: 104.4312 - val_mse: 22696.9160 - lr: 0.0010
Epoch 5/100
295750/295750 [==============================] - 310s 1ms/step - loss: 23214.1211 - mae: 106.1353 - mse: 23214.1211 - val_loss: 22622.3184 - val_mae: 103.3761 - val_mse: 22622.3184 - lr: 0.0010
Epoch 6/100
295750/295750 [==============================] - 307s 1ms/step - loss: 23119.4062 - mae: 105.8459 - mse: 23119.4062 - val_loss: 22535.2969 - val_mae: 103.9269 - val_mse: 2253

KeyboardInterrupt: 

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

# Plot MAE and MSE
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.legend()
plt.title('Mean Absolute Error Progression During Training')

plt.subplot(1, 2, 2)
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.legend()
plt.title('Mean Squared Error Progression During Training')

plt.tight_layout()
plt.show()